<a href="https://colab.research.google.com/github/jfahadi/Android-Project/blob/master/FLImg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install virtualenv
# Create a new virtual environment in the 'FedImgvenv' folder
!virtualenv FedImgvenv

# Activate the virtual environment
!source FedImgvenv/bin/activate
!pip install tensorflow tensorflow-federated
!pip install nibabel
# Install necessary packages inside the virtual environment
!pip install numpy pandas
# Install specific packages directly into the Colab environment
!pip install numpy pandas matplotlib






created virtual environment CPython3.10.12.final.0-64 in 557ms
  creator CPython3Posix(dest=/content/FedImgvenv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==24.3.1, setuptools==75.6.0, wheel==0.45.1
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator
  Using cached typing_extensions-4.5.0-py3-none-any.whl.metadata (8.5 kB)
Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
altair 5.5.

**Import Libraries**

In [18]:
import tensorflow as tf
import tensorflow_federated as tff
from tensorflow.keras import layers, models
import numpy as np
import os
from tensorflow_federated.python.learning import algorithms
from tensorflow_privacy.privacy.optimizers.dp_optimizer_keras import DPKerasSGDOptimizer
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from google.colab import drive
from google.colab import files
from sklearn.model_selection import train_test_split
import kagglehub
import tarfile
import nibabel as nib
from scipy.ndimage import zoom

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


**Load Data**

In [6]:
#Download latest version
path = kagglehub.dataset_download("dschettler8845/brats-2021-task1")

print("Path to dataset files:", path)

#data_path = "/content/drive/MyDrive/BraTS/"


Path to dataset files: /root/.cache/kagglehub/datasets/dschettler8845/brats-2021-task1/versions/1


In [10]:
# Extract all `.tar` files in the dataset directory
def extract_tar_files(dataset_path, extract_to='/content/brats_data'):
    """
    Extract all .tar files in the dataset directory.

    Args:
        dataset_path (str): Path to the directory containing .tar files.
        extract_to (str): Directory to extract the files.

    Returns:
        str: Path to the extracted data directory.
    """
    if not os.path.exists(extract_to):
        os.makedirs(extract_to)

    for filename in os.listdir(dataset_path):
        if filename.endswith('.tar'):
            tar_path = os.path.join(dataset_path, filename)
            with tarfile.open(tar_path, 'r') as tar:
                tar.extractall(path=extract_to)
                print(f"Extracted: {tar_path} to {extract_to}")

    return extract_to

# Extract all tar files
dataset_path = '/root/.cache/kagglehub/datasets/dschettler8845/brats-2021-task1/versions/1'
extracted_path = extract_tar_files(dataset_path)

Extracted: /root/.cache/kagglehub/datasets/dschettler8845/brats-2021-task1/versions/1/BraTS2021_00495.tar to /content/brats_data
Extracted: /root/.cache/kagglehub/datasets/dschettler8845/brats-2021-task1/versions/1/BraTS2021_Training_Data.tar to /content/brats_data
Extracted: /root/.cache/kagglehub/datasets/dschettler8845/brats-2021-task1/versions/1/BraTS2021_00621.tar to /content/brats_data


**Inspect extracted files**

In [11]:

# Inspect the extracted folder structure
for root, dirs, files in os.walk(extracted_path):
    print(f"Directory: {root}")
    for filename in files:
        print(f"  File: {filename}")


Streaming output truncated to the last 5000 lines.
  File: BraTS2021_00765_t1ce.nii.gz
  File: BraTS2021_00765_flair.nii.gz
Directory: /content/brats_data/BraTS2021_00740
  File: BraTS2021_00740_flair.nii.gz
  File: BraTS2021_00740_t2.nii.gz
  File: BraTS2021_00740_seg.nii.gz
  File: BraTS2021_00740_t1ce.nii.gz
  File: BraTS2021_00740_t1.nii.gz
Directory: /content/brats_data/BraTS2021_00018
  File: BraTS2021_00018_t1.nii.gz
  File: BraTS2021_00018_flair.nii.gz
  File: BraTS2021_00018_seg.nii.gz
  File: BraTS2021_00018_t2.nii.gz
  File: BraTS2021_00018_t1ce.nii.gz
Directory: /content/brats_data/BraTS2021_01429
  File: BraTS2021_01429_seg.nii.gz
  File: BraTS2021_01429_t1ce.nii.gz
  File: BraTS2021_01429_flair.nii.gz
  File: BraTS2021_01429_t2.nii.gz
  File: BraTS2021_01429_t1.nii.gz
Directory: /content/brats_data/BraTS2021_00742
  File: BraTS2021_00742_t1.nii.gz
  File: BraTS2021_00742_flair.nii.gz
  File: BraTS2021_00742_seg.nii.gz
  File: BraTS2021_00742_t2.nii.gz
  File: BraTS2021_00

**Update the dataset loader then load and preprocess**

In [3]:
def resize_volume(volume, target_size):
    """
    Resize a 3D volume to the target size.

    Args:
        volume (numpy.ndarray): The input 3D volume.
        target_size (tuple): Desired output size (height, width, depth).

    Returns:
        numpy.ndarray: Resized 3D volume.
    """
    current_size = volume.shape
    zoom_factors = [target_size[i] / current_size[i] for i in range(3)]
    resized_volume = zoom(volume, zoom_factors, order=1)  # Linear interpolation
    return resized_volume

def load_and_preprocess_data(dataset_path, image_size=(128, 128, 128), modality="flair"):
    """
    Load and preprocess the BraTS dataset for a given modality.

    Args:
        dataset_path (str): Path to the directory containing the extracted BraTS dataset.
        image_size (tuple): Target image size (height, width, depth).
        modality (str): Desired modality to use for training (e.g., 't1', 't1ce', 't2', 'flair').

    Returns:
        Tuple: Train and test datasets (x_train, y_train), (x_test, y_test).
    """
    images = []
    masks = []

    # Iterate through each subject's directory
    for subject_dir in sorted(os.listdir(dataset_path)):
        subject_path = os.path.join(dataset_path, subject_dir)
        if os.path.isdir(subject_path):
            # Find the modality and segmentation files
            modality_file = os.path.join(subject_path, f"{subject_dir}_{modality}.nii.gz")
            segmentation_file = os.path.join(subject_path, f"{subject_dir}_seg.nii.gz")

            # Ensure both files exist
            if os.path.exists(modality_file) and os.path.exists(segmentation_file):
                # Load modality and segmentation files
                img = nib.load(modality_file).get_fdata()
                mask = nib.load(segmentation_file).get_fdata()

                # Resize to target size
                img_resized = resize_volume(img, image_size)
                mask_resized = resize_volume(mask, image_size)

                # Normalize the image to [0, 1]
                img_resized = img_resized / np.max(img_resized)

                # Append to lists
                images.append(img_resized)
                masks.append(mask_resized)

    # Convert to numpy arrays and add channel dimension
    images = np.expand_dims(np.array(images, dtype=np.float32), -1)
    masks = np.expand_dims(np.array(masks, dtype=np.float32), -1)

    # Split into training and testing datasets
    x_train, x_test, y_train, y_test = train_test_split(images, masks, test_size=0.2, random_state=42)

    return (x_train, y_train), (x_test, y_test)


NameError: name 'os' is not defined

**Load and Pre-process dataset**

In [17]:

# Path to the dataset
dataset_path = "/content/brats_data"

# Load and preprocess the data
(x_train, y_train), (x_test, y_test) = load_and_preprocess_data(dataset_path, modality="flair")

# Output the shapes for verification
print(f"Training data shape: {x_train.shape}, {y_train.shape}")
print(f"Testing data shape: {x_test.shape}, {y_test.shape}")

FileNotFoundError: [Errno 2] No such file or directory: '/root/.cache/kagglehub/datasets/dschettler8845/brats-2021-task1/versions/1/imagesTr'

**Simulate Federated Data**

In [ ]:
def simulate_institution_data(x_train, y_train, num_institutions=5):
    """
    Split training data into subsets to simulate multiple institutions.

    Args:
        x_train (numpy.ndarray): Training images.
        y_train (numpy.ndarray): Training masks.
        num_institutions (int): Number of institutions to simulate.

    Returns:
        List: Data subsets for each institution.
    """
    data_per_institution = len(x_train) // num_institutions
    institutions_data = [
        (x_train[i * data_per_institution:(i + 1) * data_per_institution],
         y_train[i * data_per_institution:(i + 1) * data_per_institution])
        for i in range(num_institutions)
    ]
    return institutions_data

# Simulate federated data
institution_data = simulate_institution_data(x_train, y_train)

# Verify institution data shapes
for i, (x, y) in enumerate(institution_data):
    print(f"Institution {i+1} data shape: {x.shape}, {y.shape}")


**Define the CNN Model**

In [ ]:

def create_cnn_model(input_shape=(128, 128, 128, 1)):
    """
    Create a 3D CNN model for segmentation.

    Args:
        input_shape (tuple): Shape of the input data.

    Returns:
        tensorflow.keras.Model: The compiled CNN model.
    """
    model = models.Sequential([
        layers.Conv3D(32, (3, 3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling3D((2, 2, 2)),
        layers.Conv3D(64, (3, 3, 3), activation='relu'),
        layers.MaxPooling3D((2, 2, 2)),
        layers.Conv3D(128, (3, 3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')  # Segmentation output
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


**Federated Learning Framework**

In [ ]:

def model_fn():
    """
    Create a model function for federated learning.

    Returns:
        tff.learning.Model: The TFF-compatible model.
    """
    model = create_cnn_model()
    input_spec = {
        'x': tf.TensorSpec(shape=(None, 128, 128, 128, 1), dtype=tf.float32),
        'y': tf.TensorSpec(shape=(None, 128, 128, 128, 1), dtype=tf.float32),
    }
    return tff.learning.from_keras_model(
        keras_model=model,
        input_spec=input_spec,
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

# Federated averaging process
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.01),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0)
)

state = iterative_process.initialize()

# Train for multiple rounds
NUM_ROUNDS = 10
for round_num in range(1, NUM_ROUNDS + 1):
    state, metrics = iterative_process.next(state, institution_data)
    print(f"Round {round_num}: {metrics}")


**Evaluate the Model**

In [ ]:
def evaluate_federated_model(state, test_data):
    """
    Evaluate the federated model on test data.

    Args:
        state: The final state of the federated model.
        test_data (list): Test dataset.

    Returns:
        float: Aggregated accuracy.
    """
    aggregated_accuracy = 0
    for x, y in test_data:
        client_model = tff.learning.assign_weights_to_keras_model(state.model, model_fn())
        loss, accuracy = client_model.evaluate(x, y)
        aggregated_accuracy += accuracy
    return aggregated_accuracy / len(test_data)

# Evaluate the federated model
federated_test_accuracy = evaluate_federated_model(state, [(x_test, y_test)])
print(f"Federated Test Accuracy: {federated_test_accuracy}")
